In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *
import numpy as np
import s3fs
import io

from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.feature import VectorAssembler, StandardScaler
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator
import matplotlib.pyplot as plt
import seaborn as sns
import subprocess
import boto3
from botocore.config import Config 
from typing import Tuple,List
import sys
import socket
from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf

In [3]:
MASTER_IP = "192.168.128.236"  # IP del Server Master
MINIO_BUCKET_NAME="satellite-data"
MINIO_ADDRESS="http://192.168.128.236:9000"
# Configurazione MinIO
MINIO_ENDPOINT = f"http://{MASTER_IP}:9000"
MINIO_ACCESS_KEY = "minioadmin"
MINIO_SECRET_KEY = "minioadmin"


## Setup Spark context 

In [4]:
# Trova automaticamente l'IP della TUA macchina (Client PC7)
# che è visibile al Master.
def get_local_ip(target_ip):
    with socket.socket(socket.AF_INET, socket.SOCK_DGRAM) as s:
        try:
            # Non invia dati, serve solo a capire quale interfaccia di rete usare
            s.connect((target_ip, 80)) 
            return s.getsockname()[0]
        except Exception:
            return "127.0.0.1"

CLIENT_IP = get_local_ip(MASTER_IP)
print(f"📡 Il mio IP Client (Driver) è: {CLIENT_IP}")
print(f"🎯 Mi connetto al Master su: {MASTER_IP}")



# -------------------------------------------------------
# 2. Pulizia sessioni precedenti (Anti-Zombie)
# -------------------------------------------------------
try:
    if 'spark' in globals(): spark.stop()
    if SparkContext._active_spark_context: SparkContext._active_spark_context.stop()
    SparkContext._active_spark_context = None
except Exception:
    pass

# -------------------------------------------------------
# 3. Inizializzazione Spark Session
# -------------------------------------------------------
conf = SparkConf()
conf.setAppName("SatStream-Distributed-Client")
conf.setMaster(f"spark://{MASTER_IP}:7077")

# CONFIGURAZIONE RETE FONDAMENTALE
# Il Driver (questo PC) deve essere raggiungibile dai Worker remoti
conf.set("spark.driver.host", CLIENT_IP)        # <--- CORRETTO: Usa l'IP del Client, non del Master
conf.set("spark.driver.bindAddress", "0.0.0.0") # Ascolta su tutte le interfacce locali
conf.set("spark.driver.port", "20000")          # Porta fissa per firewall
conf.set("spark.blockManager.port", "20001")    # Porta fissa per dati

# Configurazione S3/MinIO
conf.set("spark.hadoop.fs.s3a.endpoint", MINIO_ENDPOINT)
conf.set("spark.hadoop.fs.s3a.access.key", MINIO_ACCESS_KEY)
conf.set("spark.hadoop.fs.s3a.secret.key", MINIO_SECRET_KEY)
conf.set("spark.hadoop.fs.s3a.path.style.access", "true")
conf.set("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
conf.set("spark.hadoop.fs.s3a.connection.ssl.enabled", "false")

# Timeout e Heartbeat (utili in reti distribuite)
conf.set("spark.network.timeout", "600s")
conf.set("spark.executor.heartbeatInterval", "60s")

print("🚀 Tentativo di connessione al Cluster...")

try:
    spark = SparkSession.builder.config(conf=conf).getOrCreate()
    
    print(f"\n✅ SUCCESS! Connesso al Cluster.")
    print(f"🔗 Master: {spark.sparkContext.master}")
    print(f" Web UI (Locale): http://{CLIENT_IP}:4040")
    print(f" Web UI (Master): http://{MASTER_IP}:8080")
    
    # Test veloce
    count = spark.range(100).count()
    print(f"🧪 Test di calcolo distribuito: {count} righe contate.")
    
except Exception as e:
    print(f"\n❌ ERRORE CRITICO:\n{e}")
    print("\nSUGGERIMENTO: Controlla che la porta 7077 sul Master sia aperta e che 'spark.driver.host' sia raggiungibile dai Worker.")

📡 Il mio IP Client (Driver) è: 192.168.128.236
🎯 Mi connetto al Master su: 192.168.128.236
🚀 Tentativo di connessione al Cluster...


25/12/27 10:02:54 WARN Utils: Your hostname, PC7 resolves to a loopback address: 127.0.1.1; using 192.168.128.236 instead (on interface eno1)
25/12/27 10:02:54 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/12/27 10:02:54 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/12/27 10:02:55 WARN Utils: Service 'sparkDriver' could not bind on port 20000. Attempting port 20001.
25/12/27 10:02:55 WARN Utils: Service 'sparkDriver' could not bind on port 20001. Attempting port 20002.
25/12/27 10:02:55 WARN Utils: Service 'sparkDriver' could not bind on port 20002. Attempting port 20003.
25/12/27 10:02:55 WARN Utils: Service 'sparkDriver' could not bind on port 20003. Attempting port 20004.
25/12/27 10:02:55 WARN Utils: Service 'sparkDriver' could not bind on port 20


✅ SUCCESS! Connesso al Cluster.
🔗 Master: spark://192.168.128.236:7077
 Web UI (Locale): http://192.168.128.236:4040
 Web UI (Master): http://192.168.128.236:8080


🧪 Test di calcolo distribuito: 100 righe contate.


## Verifica dataset MinIO (boto3 + mc)

In [5]:
# 🔧 BYPASS S3 Spark → Usa mc + Python nativo (100% funziona)
print(" Lista patch via mc (conferma esistenza)")
result = subprocess.run([
    "docker", "exec", "minio", "mc", "ls", 
    "myminio/satellite-data/patches/", "|", "wc", "-l"
], shell=True, capture_output=True, text=True)

print(result.stdout)
print(" mc conferma patch esistono!")

# 🔧 S3 client Python nativo (bypassa Spark S3A)
s3 = boto3.client(
    's3',
    endpoint_url=MINIO_ADDRESS,
    aws_access_key_id=MINIO_ACCESS_KEY,
    aws_secret_access_key=MINIO_SECRET_KEY
)

response = s3.list_objects_v2(Bucket=MINIO_BUCKET_NAME, Prefix='patches/')
files = [obj['Key'] for obj in response.get('Contents', []) if obj['Key'].endswith('.npz')]
print(f" {len(files):,} patch .npz confermate via boto3!")

# Prime 10
print(" Prime 10:")
for f in files[:10]:
    size = next(obj['Size'] for obj in response['Contents'] if obj['Key'] == f)
    print(f"  {size/1024/1024:.1f}MB {f}")




 Lista patch via mc (conferma esistenza)

 mc conferma patch esistono!
 862 patch .npz confermate via boto3!
 Prime 10:
  2.3MB patches/task_0_worker_2.npz
  2.4MB patches/task_1000_worker_2.npz
  2.4MB patches/task_1001_worker_3.npz
  2.6MB patches/task_1004_worker_9.npz
  2.4MB patches/task_1011_worker_3.npz
  2.4MB patches/task_1012_worker_9.npz
  2.5MB patches/task_1016_worker_3.npz
  2.4MB patches/task_1017_worker_2.npz
  0.9MB patches/task_1018_worker_3.npz
  1.7MB patches/task_101_worker_9.npz


## Estrai pixel + Feature Engineering Sentinel-2

In [6]:
# ORDINE FINALE dopo stackstac.stack() (ALFABETICO!)
BANDS_ORDER = {
    0: 'blue',      # B02 (490nm)
    1: 'green',     # B03 (560nm)  
    2: 'nir',       # B08 (842nm) ← NIR principale
    3: 'nir08',     # B8A (865nm) ← NIR stretto
    4: 'red',       # B04 (665nm)
    5: 'rededge1',  # B05 (705nm)
    6: 'rededge2',  # B06 (740nm)
    7: 'rededge3',  # B07 (783nm)
    8: 'swir16',    # B11 (1610nm) ← SWIR1
    9: 'swir22'     # B12 (2190nm) ← SWIR2
}


In [7]:

def extract_pixels(s3_path: str) -> List[Tuple[List[float], float]]:
    """
    Estrae pixel features Sentinel-2 + labels da file .npz su MinIO S3.
    
    Args:
        s3_path: Path S3 patch .npz

    Output features (6 totali per pixel):
        | Index | Feature     | Banda   | Descrizione                  |
        |-------|-------------|---------|------------------------------|
        | 0     | Red (B04)   | bands[3]| Riflettanza Rossa (665nm)   |
        | 1     | NIR (B08)   | bands[7]| Infrarosso Vicino (842nm)   |
        | 2     | SWIR1(B11)  | bands[10]| Infrarosso SWIR (1610nm)   |
        | 3     | NDVI        | -       | (NIR-Red)/(NIR+Red)         |
        | 4     | NDWI        | -       | (Green-SWIR)/(Green+SWIR)   |
        | 5     | NDMI        | -       | (NIR-SWIR)/(NIR+SWIR)       |
    
    Returns:
        [(features_6d, label), ...] dove features = [Red, NIR, SWIR1, NDVI, NDWI, NDMI]
        - features_6d: [Red, NIR, SWIR1, NDVI, NDWI, NDMI]
        - label: 0.0 (non-crop) o 1.0 (crop)
    """
    try:
        fs = s3fs.S3FileSystem(
            key=MINIO_ACCESS_KEY, secret=MINIO_SECRET_KEY,
            client_kwargs={'endpoint_url': MINIO_ADDRESS}
        )
        with fs.open(s3_path.replace('s3a://', ''), 'rb') as f:
            data = np.load(io.BytesIO(f.read()))
        
        bands = data['bands']  # (10, H, W)
        mask = data['mask'].astype(float)
        
        # 🌱 INDICI CORRETTI stackstac alfabetico
        b3_green = bands[1].flatten().astype(np.float32)   # 'green' → index 1
        b4_red = bands[4].flatten().astype(np.float32)     # 'red'   → index 4
        b8_nir = bands[2].flatten().astype(np.float32)     # 'nir'   → index 2
        b11_swir = bands[8].flatten().astype(np.float32)   # 'swir16'→ index 8
        
        # 🌿 FIX: SENZA out= parameter (crea nuovo array float)
        ndvi = np.where(
            (b8_nir + b4_red) != 0,
            (b8_nir - b4_red) / (b8_nir + b4_red + 1e-8),
            0.0
        )
        
        ndwi = np.where(
            (b3_green + b11_swir) != 0,
            (b3_green - b11_swir) / (b3_green + b11_swir + 1e-8),
            0.0
        )
        
        ndmi = np.where(
            (b8_nir + b11_swir) != 0,
            (b8_nir - b11_swir) / (b8_nir + b11_swir + 1e-8),
            0.0
        )
        
        # 6 features finali
        features = np.column_stack([
            b4_red, b8_nir, b11_swir,  # Raw bands
            ndvi, ndwi, ndmi           # Spectral indices
        ])
        
        labels = mask.flatten()
        
        # Filtra pixel validi (0 o 1)
        valid = (labels == 0) | (labels == 1)
        return [(features[valid][i].tolist(), float(labels[valid][i]))
                for i in range(len(labels[valid]))]
        
    except Exception as e:
        print(f"❌ Error {s3_path}: {e}")
        import traceback
        traceback.print_exc()
        return []

# UDF Spark
schema = ArrayType(StructType([
    StructField("features", ArrayType(FloatType()), False),
    StructField("label", FloatType(), False)
]))
extract_udf = udf(extract_pixels, schema)

print("✅ UDF fixed (np.where instead of np.divide out=)")


✅ UDF fixed (np.where instead of np.divide out=)


In [8]:
# %% TEST: Conferma ordine bande
first_file = files[0]
obj = s3.get_object(Bucket='satellite-data', Key=first_file)
data = np.load(io.BytesIO(obj['Body'].read()))

print(f"📊 bands.shape: {data['bands'].shape}")  # (10, 256, 256)

# Stampa statistiche per identificare bande
for i in range(10):
    band = data['bands'][i]
    print(f"  Band {i}: mean={band.mean():.1f} std={band.std():.1f} "
          f"range=[{band.min()}, {band.max()}]")

# Verifica NDVI
red = data['bands'][4]
nir = data['bands'][2]
ndvi = (nir - red) / (nir + red + 1e-8)
print(f"\n🌿 NDVI range: [{ndvi.min():.3f}, {ndvi.max():.3f}]")
print(f"   (atteso: -0.2 → +0.8 per vegetazione)")


📊 bands.shape: (10, 581, 443)
  Band 0: mean=213.9 std=123.3 range=[1, 3284]
  Band 1: mean=396.3 std=165.3 range=[33, 3480]
  Band 2: mean=364.9 std=199.1 range=[21, 3836]
  Band 3: mean=2379.8 std=571.7 range=[470, 5688]
  Band 4: mean=786.8 std=238.9 range=[149, 4039]
  Band 5: mean=1885.6 std=426.3 range=[553, 4255]
  Band 6: mean=2266.7 std=503.8 range=[753, 5003]
  Band 7: mean=2571.5 std=546.3 range=[991, 5495]
  Band 8: mean=1765.4 std=548.4 range=[523, 5081]
  Band 9: mean=914.0 std=395.2 range=[220, 3784]

🌿 NDVI range: [0.000, 342.560]
   (atteso: -0.2 → +0.8 per vegetazione)


## 📊 CELL 5: Estrai pixel → DataFrame 6 colonne


### STEP 1: Lista patch S3 (100 patch per test veloce)

In [9]:
# %% [markdown]
# # ⚡ VELOCIZZATO: Cache + 1 solo count finale

# %%
import time
start = time.time()

# STEP 1-2: Paths (invariato)
s3 = boto3.client('s3', endpoint_url=MINIO_ADDRESS, 
                  aws_access_key_id=MINIO_ACCESS_KEY, aws_secret_access_key=MINIO_SECRET_KEY)

response = s3.list_objects_v2(Bucket='satellite-data', Prefix='patches/')
files = [obj['Key'] for obj in response['Contents'] if obj['Key'].endswith('.npz')][:100]
paths = ['s3a://satellite-data/' + f for f in files]

print(f"🎯 {len(paths)} patch selezionate")

patches_df = spark.createDataFrame(
    [(path,) for path in paths],
    StructType([StructField("path", StringType())])
).repartition(20)  # ← Parallelizza su 20 partizioni

# STEP 3: UDF + CACHE (esegui subito)
print("🔄 Esecuzione UDF (può richiedere 5-10 min)...")
pixels_raw_df = patches_df.withColumn("pixels", extract_udf(col("path"))) \
    .cache()  # ← CACHE risultato UDF!

# Force esecuzione UDF ADESSO (1 volta sola)
n_patches = pixels_raw_df.count()
print(f"✅ {n_patches} patch processate in {time.time()-start:.1f}s")

# STEP 4: Esplodi (veloce, usa cache)
pixels_df = pixels_raw_df.select(explode("pixels").alias("pixel"))

# STEP 5: 6 colonne + CACHE
features_df = pixels_df.select(
    col("pixel.features")[0].alias("red_b4"),
    col("pixel.features")[1].alias("nir_b8"),
    col("pixel.features")[2].alias("swir_b11"),
    col("pixel.features")[3].alias("ndvi"),
    col("pixel.features")[4].alias("ndwi"),
    col("pixel.features")[5].alias("ndmi"),
    col("pixel.label").alias("label")
).cache()  # ← CACHE features finali

# 1 SOLO COUNT (esegue explode + select)
total_pixels = features_df.count()
print(f"✅ {total_pixels:,} pixel in {time.time()-start:.1f}s totali")

# Statistiche (usano cache)
features_df.groupBy("label").count().orderBy("label").show()
features_df.describe("ndvi", "label").show()
features_df.show(5)

print(f"\n⏱️  Tempo totale: {time.time()-start:.1f}s ({(time.time()-start)/60:.1f} min)")


🎯 100 patch selezionate
🔄 Esecuzione UDF (può richiedere 5-10 min)...


25/12/27 10:03:26 WARN TaskSetManager: Lost task 3.0 in stage 5.0 (TID 14) (127.0.0.1 executor 1): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/opt/spark/python/lib/pyspark.zip/pyspark/worker.py", line 1231, in main
    func, profiler, deserializer, serializer = read_udfs(pickleSer, infile, eval_type)
  File "/opt/spark/python/lib/pyspark.zip/pyspark/worker.py", line 1067, in read_udfs
    udfs.append(read_single_udf(pickleSer, infile, eval_type, runner_conf, udf_index=i))
  File "/opt/spark/python/lib/pyspark.zip/pyspark/worker.py", line 529, in read_single_udf
    f, return_type = read_command(pickleSer, infile)
  File "/opt/spark/python/lib/pyspark.zip/pyspark/worker.py", line 90, in read_command
    command = serializer._read_with_length(file)
  File "/opt/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 174, in _read_with_length
    return self.loads(obj)
  File "/opt/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 4

PythonException: 
  An exception was thrown from the Python worker. Please see the stack trace below.
Traceback (most recent call last):
  File "/opt/spark/python/lib/pyspark.zip/pyspark/worker.py", line 1231, in main
    func, profiler, deserializer, serializer = read_udfs(pickleSer, infile, eval_type)
  File "/opt/spark/python/lib/pyspark.zip/pyspark/worker.py", line 1067, in read_udfs
    udfs.append(read_single_udf(pickleSer, infile, eval_type, runner_conf, udf_index=i))
  File "/opt/spark/python/lib/pyspark.zip/pyspark/worker.py", line 529, in read_single_udf
    f, return_type = read_command(pickleSer, infile)
  File "/opt/spark/python/lib/pyspark.zip/pyspark/worker.py", line 90, in read_command
    command = serializer._read_with_length(file)
  File "/opt/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 174, in _read_with_length
    return self.loads(obj)
  File "/opt/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 472, in loads
    return cloudpickle.loads(

25/12/27 10:03:30 WARN TaskSetManager: Lost task 4.0 in stage 5.0 (TID 15) (192.168.128.232 executor 0): TaskKilled (Stage cancelled: Job aborted due to stage failure: Task 1 in stage 5.0 failed 4 times, most recent failure: Lost task 1.3 in stage 5.0 (TID 34) (127.0.0.1 executor 1): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/opt/spark/python/lib/pyspark.zip/pyspark/worker.py", line 1231, in main
    func, profiler, deserializer, serializer = read_udfs(pickleSer, infile, eval_type)
  File "/opt/spark/python/lib/pyspark.zip/pyspark/worker.py", line 1067, in read_udfs
    udfs.append(read_single_udf(pickleSer, infile, eval_type, runner_conf, udf_index=i))
  File "/opt/spark/python/lib/pyspark.zip/pyspark/worker.py", line 529, in read_single_udf
    f, return_type = read_command(pickleSer, infile)
  File "/opt/spark/python/lib/pyspark.zip/pyspark/worker.py", line 90, in read_command
    command = serializer._read_with_length(file)
  File "/opt

## ML Pipeline + Training

In [ ]:
# Vector Assembler (8 features → vector)
assembler = VectorAssembler(
    inputCols=["features"], 
    outputCol="features_vec",
    handleInvalid="skip" # <-- salta le righe che contengono null
)

scaler = StandardScaler(
    inputCol="features",
    outputCol="scaled_features", # La nuova colonna vettoriale
    withMean=True,
    withStd=True
)


# Random Forest (bilanciato per CROP raro)
rf = RandomForestClassifier(
    featuresCol="features_vec",
    labelCol="label",
    numTrees=100,
    maxDepth=8,
    impurity="gini",
    featureSubsetStrategy="sqrt"
)

# SPLIT 80/20
train_df, test_df = pixels_df.randomSplit([0.8, 0.2], seed=42)

print(f"📊 Train: {train_df.count():,} | Test: {test_df.count():,} pixel")

# TRAIN
train_assembled = assembler.transform(train_df)
model = rf.fit(train_assembled)

print("✅ Modello addestrato!")


📊 Train: 0 | Test: 0 pixel


IllegalArgumentException: Data type array<float> of column features is not supported.

In [ ]:
# PREDIZIONI
test_assembled = assembler.transform(test_df)
predictions = model.transform(test_assembled)

# Metriche
roc_evaluator = BinaryClassificationEvaluator(
    labelCol="label", rawPredictionCol="rawPrediction", metricName="areaUnderROC"
)
auc = roc_evaluator.evaluate(predictions)
print(f"🎯 AUC-ROC: {auc:.4f}")

f1_evaluator = MulticlassClassificationEvaluator(
    labelCol="label", predictionCol="prediction", metricName="f1"
)
f1 = f1_evaluator.evaluate(predictions)
print(f"📊 F1-Score: {f1:.4f}")

# Confusion Matrix
cm_df = predictions.groupBy("label", "prediction").count().toPandas()
print("🔢 Confusion Matrix:")
print(cm_df)

# Feature Importance
importances = model.featureImportances
print("\n🌿 Feature Importance:")
features = ["Red", "NIR", "SWIR1", "NDVI", "NDWI", "NDMI"]
for i, (feat, imp) in enumerate(zip(features, importances)):
    print(f"  {feat}: {imp:.3f}")


In [ ]:
# Salva modello su MinIO
model.write().overwrite().save("s3a://satellite-data/models/crop_classifier_rf_v1")
print("✅ Modello salvato su MinIO!")

# Test su 1 patch nuova
test_patch = patches_df.filter(col("path").contains("task_0")).first()
test_pixels = spark.createDataFrame(
    extract_pixels(test_patch.path), 
    schema
).select("features", "label")

test_assembled = assembler.transform(test_pixels)
test_pred = model.transform(test_assembled)

print("🧪 Test nuova patch:")
test_pred.groupBy("label", "prediction").count().show()


In [ ]:
spark.stop()
print("✅ Spark terminato!")